In [1]:
from Bio import Entrez, SeqIO
from tqdm import tqdm
import pandas as pd
import time

In [ ]:
Entrez.email = "your_email@example.com"

# 개별 검색

In [12]:
data = pd.read_csv("../train_data/merge_train_preprocess_refine_columns.csv")

In [9]:
# 검색할 단백질 이름을 입력합니다
protein_name = "BRCA2"
# Step 1: 단백질 이름으로 NCBI protein 데이터베이스 검색
try:
    handle = Entrez.esearch(db="protein", term=protein_name, retmax=1)
    record = Entrez.read(handle)
    protein_ids = record["IdList"]

    # 단백질 ID가 존재할 경우에만 서열을 가져옵니다
    if protein_ids:
        protein_id = protein_ids[0]
        print(f"Found Protein ID: {protein_id}")
        
        # Step 2: 단백질 서열 가져오기
        handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
        seq_record = SeqIO.read(handle, "fasta")
        
        # 단백질 서열 출력
        print("Protein Name:", protein_name)
        print("Description:", seq_record.description)
        print("Sequence:", seq_record.seq)
    else:
        print("No protein found with the given name.")
except Exception as e:
    print(f"An error occurred: {e}")


Found Protein ID: 2813727314
Protein Name: BRCA2
Description: XP_068818217.1 lys-63-specific deubiquitinase BRCC36 [Capricornis sumatraensis]
Sequence: MAVQVVQAVQAVHLESDAFLVCLNHALSTEKEEVMGLCIGELNDDLRNDPKFTYTGTEMRTVAEKVDTVRIVHIHSVIILRRSDKRKDRVEISPEQLSAASTEAERLAELTGRPMRVVGWYHSHPHITVWPSHVDVRTQAMYQMMDQGFVGLIFSCFIEDKNTKTGRVLYTCFQSIQAQKSSESPRGPRDFWSSSQHISIEGQKEEERYERIEIPIHIVPHVTIGKVCLESAVELPKILCQEEQDAYRRIHSLTHLDSVTKIHNGSVFTKNLCSQMSAVSGPLLQWLEDRLEQNQQHVQELQQEKEELLQELSSLE


# refine_col로 추출

In [ ]:
# 단백질 이름 컬럼들을 추출
protein_columns = data.columns[2:]  # ID와 SUBCLASS 열 제외

# 단백질 서열을 저장할 딕셔너리 생성
protein_sequences = {protein: [] for protein in protein_columns}
non_seq = []
# 각 단백질에 대해 NCBI에서 서열 검색 후 딕셔너리에 추가
for protein in tqdm(protein_columns):
    try:
        # NCBI protein 데이터베이스에서 검색
        handle = Entrez.esearch(db="protein", term=protein, retmax=1)
        record = Entrez.read(handle)
        protein_ids = record["IdList"]

        # 단백질 ID가 존재하는 경우 서열 가져오기
        if protein_ids:
            protein_id = protein_ids[0]
            handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
            seq_record = SeqIO.read(handle, "fasta")
            protein_sequences[protein] = seq_record.seq
        else:
            protein_sequences[protein] = None
            print(f"{protein}은 서열 정보가 없습니다.")
            non_seq.append(protein)

        # 너무 많은 요청을 보내지 않도록 잠시 대기
        time.sleep(1)
    except Exception as e:
        protein_sequences[protein] = None
        print(f"Error fetching sequence for {protein}: {e}")



  0%|          | 0/148 [00:00<?, ?it/s]

 81%|████████  | 120/148 [10:26<01:32,  3.32s/it]

In [ ]:
# DataFrame에 단백질 서열 추가
df_sample = pd.DataFrame([data])
df_sample.head()

In [ ]:
# 최종 결과를 CSV 파일로 저장
df_sample.to_csv('../train_data/protein_sequences_output_sample_refine_col.csv', index=False)


# 전체 col로 추출

In [ ]:
train = pd.read_csv("../train_data/train.csv")

In [ ]:
# 단백질 이름 컬럼들을 추출
protein_columns = data.columns[2:]  # ID와 SUBCLASS 열 제외

# 단백질 서열을 저장할 딕셔너리 생성
protein_sequences = {protein: [] for protein in protein_columns}
non_seq = []
# 각 단백질에 대해 NCBI에서 서열 검색 후 딕셔너리에 추가
for protein in tqdm(protein_columns):
    try:
        # NCBI protein 데이터베이스에서 검색
        handle = Entrez.esearch(db="protein", term=protein, retmax=1)
        record = Entrez.read(handle)
        protein_ids = record["IdList"]

        # 단백질 ID가 존재하는 경우 서열 가져오기
        if protein_ids:
            protein_id = protein_ids[0]
            handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
            seq_record = SeqIO.read(handle, "fasta")
            protein_sequences[protein] = seq_record.seq
        else:
            protein_sequences[protein] = None
            print(f"{protein}은 서열 정보가 없습니다.")
            non_seq.append(protein)

        # 너무 많은 요청을 보내지 않도록 잠시 대기
        time.sleep(1)
    except Exception as e:
        protein_sequences[protein] = None
        print(f"Error fetching sequence for {protein}: {e}")



In [ ]:
# DataFrame에 단백질 서열 추가
df = pd.DataFrame([data])
df.head()

In [ ]:

# 최종 결과를 CSV 파일로 저장
df.to_csv('../train_data/protein_sequences_output.csv', index=False)